In [1]:
from __future__ import division
import AudioLoad
import AudioFeatures
import AudioActDet
import AudioSegmentation
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import time
import pandas as pd
%matplotlib inline
import datetime

In [2]:
def convert_hs(minlist, date, dateformat):
    newdatetime = []
    for t in minlist:
        minute = int(t)
        if minute == 60:
            minutes = 0
            seconds = (t * 60) % 60
            newt = "%d:%02d" % (minutes, seconds)
            strtime = date + newt
            s_datetime = datetime.datetime.strptime(strtime, dateformat + '%M:%S') + datetime.timedelta(hours = 1)
        else:
            minutes = minute
            seconds = (t * 60) % 60
            newt = "%d:%02d" % (minutes, seconds)
            strtime = date + newt
            s_datetime = datetime.datetime.strptime(strtime, dateformat + '%M:%S')
        newdatetime.append(s_datetime)
    return newdatetime

In [3]:
reload(AudioLoad)
reload(AudioFeatures)
reload(AudioActDet)

<module 'AudioActDet' from 'AudioActDet.pyc'>

In [4]:
path = "/Users/dl/GSR/Audio/"
files = [path + i for i in os.listdir(path) 
         if os.path.isfile(os.path.join(path,i)) and 'KJFK-Twr-Apr-28' in i]

In [ ]:
for j in files:
    AudioClass = AudioLoad.AudioLoad(file_list = [j])
    FeatureClass = AudioFeatures.AudioFeatures(AudioLoad = AudioClass,
                                       nperseg = 512,
                                       overlap_rate = 8, 
                                       nfft = 1024, 
                                       fbank_hfreq = None,
                                       pre_emphasis = True)
    features = FeatureClass.FeatureExtraction(highest_order = 2, 
                                      num_cep = 12, 
                                      lifting = True, 
                                      lifting_coef = 22, 
                                      mean_normalization = True, 
                                      nshift = 2)
    filename = j.replace('/Users/dl/GSR/Audio/', '').replace('.mp3','')
    pickle.dump(FeatureClass, open('/Users/dl/GSR/Audio/Feature/' + filename + '.pkl', 'wb'), protocol=2)
    np.save('/Users/dl/GSR/Audio/Feature/' + filename, features)

In [ ]:
#np.load('/Users/dl/GSR/Audio/Feature/KJFK-Twr-Apr-28-2017-1800Z.npy')
#pickle.load(open('/Users/dl/GSR/Audio/train_201704281830.pkl', 'rb'))

In [ ]:
def vad_xmin(audio_file, time_window = 5):
    freq_frame = []
    for j in files:
        AudioClass = AudioLoad.AudioLoad(file_list = [j])
        FeatureClass = AudioFeatures.AudioFeatures(AudioLoad = AudioClass,
                                           nperseg = 512,
                                           overlap_rate = 8, 
                                           nfft = 1024, 
                                           fbank_hfreq = None,
                                           pre_emphasis = True)
        features = FeatureClass.FeatureExtraction(highest_order = 2, 
                                          num_cep = 12, 
                                          lifting = True, 
                                          lifting_coef = 22, 
                                          mean_normalization = True, 
                                          nshift = 2)
        VADClass = AudioActDet.AudioActDet(FeatureClass)
        sil_seg, silence_seg_2d, idx_act = VADClass.detect_silence(power_threshold = 0,
                                             silence_sec = 0.25, 
                                             mvg_point = 5)

        # Create 5-min time interval table
        currtime  = datetime.datetime.strptime(j[47:63], '%b-%d-%Y-%H%M')
        sta_list = [currtime + datetime.timedelta(minutes=time_window*x) for x in range(0, int(30/time_window))]
        end_list = [currtime + datetime.timedelta(minutes=time_window*x) for x in range(1, int(30/time_window + 1))]
        freq = pd.DataFrame({'ST': sta_list, 'ED':end_list})

        act_rates = []
        # Get VAD result
        sil_re = silence_seg_2d/VADClass.sec_to_bin
        start = [i[1] for i in sil_re if i[1] <= 1800]
        end = [i[0] for i in sil_re if i[0] <= 1800]
        if len(start) == 0:
            act_rates = [0] * int(30/time_window)
        else:
            end.pop(0)
            if len(end) < len(start):
                end.append(min(1800, FeatureClass.sound_length))

            if j[61:63] == '00':
                st = [k/60 for k in start]
                ed = [k/60 for k in end]
            else:
                st = [k/60+30 for k in start]
                ed = [k/60+30 for k in end]

            on = convert_hs(st,j[47:61],'%b-%d-%Y-%H')
            off = convert_hs(ed,j[47:61],'%b-%d-%Y-%H')

            vad = pd.DataFrame({'Off': off, 'On': on})

            for idx, row in freq.iterrows():
                dat_stime = vad[(vad['On'] < row.ED) & (vad['Off'] > row.ST)]
                if len(dat_stime) == 0:
                    validlen = min(1800, FeatureClass.sound_length)
                    validlen_t = convert_hs([int(j[61:63]) + validlen/60],j[47:61],'%b-%d-%Y-%H')[0]
                    if row.ST <= validlen_t:
                        act_rate = 0
                    else:
                        act_rate = -1
                        print row.ST
                else:
                    dat_stime = dat_stime.sort_values(by=['On'])
                    if dat_stime.iloc[0].On < row.ST:
                        dat_stime.iloc[0].On = row.ST
                    if dat_stime.iloc[-1].Off > row.ED:
                        dat_stime.iloc[-1].Off = row.ED

                    dat_stime['Duration'] = dat_stime['Off'] - dat_stime['On']
                    act_rate = dat_stime['Duration'].sum() / datetime.timedelta(minutes = time_window)
                act_rates.append(act_rate)

        freq['AR'] = act_rates
        freq = freq[['ST', 'ED', 'AR']]
        freq_frame.append(freq)
    return freq_frame

In [ ]:
mon_list = ['Jul','Dec']
path = "/Volumes/NewVolume/ATCAudioData/Tower/"

In [ ]:
# -1 means that we don't have the audio tape available for that paticular time. 
 # And we can't decide whether it's 0 active rate. The time will print out.
# 0 means that for this period, no one talks.
for j in mon_list:
    files = [path + i for i in os.listdir(path)
             if os.path.isfile(os.path.join(path,i)) and j in i]
    ar = pd.concat(vad_xmin(files, time_window = 1),ignore_index=True)
    ar.to_csv('/Users/dl/GSR/Audio/VAD1min_' + j + '.csv', index = False)